# Read and process the central heating data 
Differentation between consumption for heating and tapwater


In [3]:
import io
import os
import sys
import types
import numpy as np
import pandas as pd
import altair as alt
import dropbox

In [4]:
%run MakeNBImportAvailable.ipynb

In [5]:
from datetime import date
from datetime import datetime, timedelta
import RetrieveData as rd_gn
import knmi_gn_001 as gethdd
import Process_Data_CH as prcs
import GetMySQL

importing Jupyter notebook from RetrieveData.ipynb
importing Jupyter notebook from knmi_gn_001.ipynb
importing Jupyter notebook from Process_Data_CH.ipynb
importing Jupyter notebook from GetMySQL.ipynb


In [8]:
def plot_Data(df,df_hdd=None,showplot=None,db_upload = None,gn_drpbx=None):
     # df : consumption for specified day (dag), can be empty.
    # dag : specified day
    # df_hdd: full dataset with daily hdd and daily gas consumption
    
    alt.data_transformers.disable_max_rows()
    
    palette = alt.Scale(domain=['abs_CVout','diff_CV', 'tapwater','abs_tapwater', 'm3_cv', 'm3_tap','totaal'],
                  range=["#1696d2", "#d2d2d2","#000000", "#fdbf11", "#ec008b", "#55b748", "#5c5859","#db2b27"])   
    
    df = df.reset_index()
    x_start = df['datetime'].min()
    x_end = df['datetime'].max()
    dag = x_start.date()


    # Chart Temperatures during the day, line & points

    source = df
    source = pd.melt(source, id_vars=['datetime'], value_vars=['abs_tapwater','abs_CVout'])
    customtitle = 'Temperaturen ' + dag.strftime('%A') + " " + str(dag)
    chart1a = alt.Chart(source,title=customtitle).mark_line(
        strokeWidth=1
    ).encode(
        x = alt.X('datetime',axis=alt.Axis(title="Tijdstip")),
        y = alt.Y("value:Q",axis=alt.Axis(title="Temperatuur [dC]")),
        color = alt.Color("variable",scale = palette)
    ).properties(width=800,height=100)
    
    # add points of heating
    source1b = df[['datetime','abs_tapwater','tap_heat']].copy()
    source1b['variable'] = 'abs_tapwater'
    source1b.columns = ['datetime','value','heat_bool','variable']
    
    source1b_2 = df[['datetime','abs_CVout','cv_heat']].copy()
    source1b_2['variable'] = 'abs_CVout'
    source1b_2.columns = ['datetime','value','heat_bool','variable']
    source1b = pd.concat([source1b,source1b_2],sort=True)
    source1b = source1b[source1b['heat_bool']==1]


    chart1b = alt.Chart(source1b.reset_index(),title=customtitle).mark_point(
        strokeWidth=1
    ).encode(
        x=alt.X('datetime',axis=alt.Axis(title="Tijdstip")),
        y=alt.Y("value:Q"),
        color = alt.Color("variable",scale = palette)
    ).properties(width=800,height=100)

    chart1 = chart1a + chart1b
    
    
    # Chart2 : Gas consumption 

    source2 = df[['datetime','m3_cv','m3_tap']].set_index('datetime').resample('15min').sum()
    #source2 = df[['datetime','m3_cv','m3_tap']].set_index('datetime')
    source2 = pd.melt(source2.reset_index(),id_vars=['datetime'], value_vars=['m3_cv','m3_tap'])
    source2 = source2[source2['value']!=0]


    chart2 = alt.Chart(source2,title="Gas verbruik").mark_bar(
        strokeWidth=1
    ).encode(
        x=alt.X('datetime',axis=alt.Axis(title="Tijdstip [15 min data]"),
                scale=alt.Scale(domain=[str(x_start),str(x_end)])),
        y=alt.Y("value:Q",axis=alt.Axis(title="Gasverbruik [m3]")),
        color="variable"
    ).properties(width=800,height=100)

       # Chart 3 : cumulative door de dag

    cumulgas = df[['datetime','m3_tap','m3_cv']].set_index('datetime')
    cumulgas.fillna(0,inplace=True)
    cumulgas = cumulgas.cumsum()
    cumulgas['totaal'] = cumulgas['m3_tap'] + cumulgas['m3_cv']
    cumulgas = pd.melt(cumulgas.reset_index(),id_vars=['datetime'])

    chart3 = alt.Chart(cumulgas,title="Cumulatief gas verbruik").mark_line(
        strokeWidth=1
    ).encode(
        x=alt.X('datetime',axis=alt.Axis(title="Tijdstip")),
        y=alt.Y("value:Q",axis=alt.Axis(title="gasverbruik [m3]")),
        color="variable"
    ).properties(width=800,height=100)


    chart = chart1 & chart2 & chart3
    
    
    # Bar chart with total day consumption
    sourcebc = cumulgas[cumulgas['datetime']==x_end].copy()
    #sourcebc = pd.melt(sourcebc,id_vars=['datetime'])
    sourcebc['value'] = round(sourcebc['value'],2)
    maxvalue = sourcebc['value'].max()

    chartb = alt.Chart(sourcebc,title="Gasverbruik").mark_bar(
        strokeWidth=1
    ).encode(
        x=alt.X('variable:N',axis=alt.Axis(title="Medium")),
        y=alt.Y("value:Q",axis=alt.Axis(title="gasverbruik [m3]"),
                scale=alt.Scale(domain=[0,maxvalue*1.2])),
        color="variable"
    ).properties(width=400,height=300)

    chartb_text = chartb.mark_text(
        align='center',baseline='middle',
        dy=-10  
        ).encode(
    text='value:Q')

    chartb = chartb + chartb_text

    # Bubble chart , but sometimes HDD is not available yet
    # 
    # Show cv verbruik vs hdd 
    # Show voorgaande 5 dagen in kleur
    
    poi_start = dag - pd.Timedelta(hours = 6*24)
    ds = 8
    pw = 400 # breedte graph
    ph = 300 # hoogte graph
    x_max = 25 # axis max x
    y_max = 20 # axis max y
    titleshown = "HDD vs CVgas"

#     if df_hdd.loc[dag,['TG','FG']].sum()==0 :
#         titleshown = "KNMI data niet up to date !"


    source = df_hdd
    chart_hdd = alt.Chart(source,title=titleshown).mark_point(clip=True,color="lightgrey").encode(
        x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
        y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max)))
    ).properties(width=pw,height=ph)

    source = df_hdd[poi_start:dag].reset_index()

    cats = df_hdd['dow'].cat.categories.tolist()
    source['daysago'] = np.arange(source['dow'].count(),0,-1)-1  

    base = alt.Chart(source).mark_point(clip=True,size=100).encode(
        x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
        y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max))))

    points = base.mark_point(clip=True,size=100).encode(
        fill = alt.Fill('daysago:Q', scale = alt.Scale(scheme='plasma')),
        shape = alt.Shape('dow:N',sort=cats),
        size = alt.Size('daysago', scale=alt.Scale(range=[250, 10]))
    )

    text = base.mark_text(
        align='left',
        baseline='middle',
        dx=7
    ).encode(
        text='daysago'
    )

    chart_hdd = chart_hdd + points + text


    # Plot next to each other
    chartb = chartb | chart_hdd
    
    # Combine all charts into 1
    chart_all=alt.vconcat(chart , chartb)
    chart_all = chart_all.configure_title(
    fontSize=14,
    font='Tahoma',
    align = 'left',
    anchor='start',
    color='gray')

    if showplot: 
        display(chart_all)


    # Create Filename 
    if dag == date.today():
        s2 = "Today.png"
    else:        
        s1 = sourcebc['value']
        s2 = ['tap','cv','tot']
        s2 = [str(m)+str(n)+'_' for m,n in zip(s2,s1)]
        s2 = ''.join(s2)
        s2 = s2.replace('.','_')
        s2 = ''.join([str(dag),'_',s2[0:-1],'.png'])
        s2 = s2.replace('-','')
    chart_all.save(s2)


    # Upload to dropbox:
    if db_upload:
        
        d = dropbox.Dropbox(gn_drpbx)
        target = ''.join(['/EnergyConsumption/',s2])
        localfile = s2
        with open(localfile, 'rb') as f:
                meta = d.files_upload(f.read(), target, mode=dropbox.files.WriteMode("overwrite"))
        print("Upload image ",s2," to Dropbox")

        os.remove(localfile)
    #print("temporary File Removed from system")

    return(source1b)

    

In [9]:
def multipledays(s1):
    # verify whether string of dates contains more than one date
    return(s1.count("-")>2)

In [10]:
# def MergeConsumptionAndHDD(df_c,df_h):
#     # calculate daily consumption and merge with hdd dataset
#     # remove the erronous days
#     # return dataset : consumption, HDD
#     df = df_c[['gas_cv','gas_tap']].resample('d').sum()
#     gm_d.merge(hrd,on="gasdatetime",how='left')

In [11]:
def plot_dailyconsumption(df,df_hdd,periodstring,showthegraph=None,upload=None):
#     df = pd.read_csv("processed_gas.csv",
#                      parse_dates=['datetime'],
#                      index_col = ['datetime'])
#     df = prcs.processdata()
#     df_hdd = gethdd.retrieveHDD() 
    
    

        
#     datetime_series = pd.Series(pd.date_range(start=dag, end=enddag, freq="D")).astype(str)
    
    # Calculate daily consumption
    df_d = df[['m3_cv','m3_tap']].resample('d').sum()
    
    df_hdd  = df_hdd.merge(df_d,on = "datetime",how="right")   
    df_hdd['checksum'] = df_hdd['TG'] + df_hdd['FG']
    
    # subset the data to relevant period of interest
    df_all = df.copy()
    df = df[df['datum'].astype(str).isin(datetime_series)]
    
    

    while dag <=  enddag:
        #print(dag)
        # plot day + cons ~ hdd van preceding days.
        # need the df for dag
        # need df_hdd for all days distinct last nr days in the function
        
        nxtdag = dag + pd.Timedelta(hours = 24)
        
        
        plotconsumption(df[dag:nxtdag],dag,df_hdd,showthegraph)
        
        
        dag = dag + pd.Timedelta(hours = 24)
    
        
    return(df_all,df_d,df_hdd,datetime_series)

In [12]:
def RetrieveDailyData(fn):
    # lees de file fn in
    try:
        df = pd.read_csv(fn,
                       parse_dates=['datetime'],
                       index_col = ['datetime'])
    except:
        print("File not found ",fn)
    
    return df

In [13]:
fn = 'gas_consumption_day.csv'
df = RetrieveDailyData(fn)

In [14]:
df.tail(10)

,m3_cv,m3_tap
datetime,,
2021-12-16,6.919,0.0
2021-12-17,5.818,0.0
2021-12-18,NaN,NaN
2021-12-19,NaN,NaN
2021-12-20,NaN,NaN
2021-12-21,NaN,NaN
2021-12-22,NaN,NaN
2021-12-23,NaN,NaN
2021-12-24,NaN,NaN


Deze data zou geen "weer"data moeten bevatten. Soms crasht het proces en wordt er toch op een gegeven moment hdd data mee opgeslagen. Een volgende keer zou het script dan crashen omdat het dataframe al HDD bevat

In [15]:
# if df.loc[df.index[-1],'TG'] == 0.0:
#     poi = (df.index[-2].date()+ pd.Timedelta(hours=24)).strftime('%Y-%m-%d')
# else:
#     poi = (df.index[-1].date()+ pd.Timedelta(hours=24)).strftime('%Y-%m-%d')
# poi = str(poi)+":"
# poi

In [16]:
poi = '2021-12-25:'
# poi = str(df.index[-1].date())+':'
poi

'2021-12-25:'

In [17]:
if __name__ == "__main__":
#     fn = 'gas_consumption_day.csv'
#     df = RetrieveDailyData(fn)
#     df = df[["m3_cv","m3_tap"]]
    
# #     poi = "2021-05-24:"
#     poi = "gisteren"
    
    poi = GetMySQL.SetPOI(poi)
    
    
    ref_date_range = pd.date_range(poi['start'],poi['end'])
    ref_date_range = ref_date_range[:-1]
    v = ref_date_range.isin(df.index)
    # 

    ToUpdate = False
    if (v.all()):
        print("Daily data available")
    else:
        print("At least one datapoint is not in file : Need update")
        ToUpdate = True
    
    # Retrieve the minute data for the poi
    # poi = "2020-12-29:2021-01-06"
    pct = 90
    df_GandT = GetMySQL.retrieveGasandTemps(poi,pct)
    df_GandT = prcs.address_consumer(df_GandT)
    

    # Process the data: bereken gasverbruik voor CV en tapwater respectievelijk
    df2 = prcs.count_stream(df_GandT)
    #     tellertje.head()
    gm = prcs.gasmomenten(df2,df_GandT)
    #     gm.tail()
    # Add the moments of the addressed consumer of gas consumption 
    df_GandT = df_GandT.reset_index().merge(gm[['gasdatetime_x','gas_cv','gas_tap','m3_cv','m3_tap']],left_on='datetime',right_on='gasdatetime_x',how='left')
    df_GandT = df_GandT.set_index('datetime')
    df_GandT.drop(['gasdatetime_x'],inplace=True,axis=1)
    sumday = df_GandT[['m3_cv','m3_tap']].resample("D").sum()
    # Remove the 0s
    sumday.loc[(sumday["m3_cv"]==0)&(sumday["m3_tap"]==0),['m3_cv']]=np.nan
    sumday.loc[(sumday["m3_cv"].isnull())&(sumday["m3_tap"]==0),['m3_tap']]=np.nan
    
    # emptydates = ("datetime" = ref_date_range[~ref_date_range.isin(sumday.index)])]
    dates2add = ref_date_range[~ref_date_range.isin(sumday.index)]
    empties = np.empty((len(dates2add),2))
    empties[:] = np.NaN
    empties = pd.DataFrame(empties)
    empties['datetime'] = dates2add
    # empties.columns = ['m3_cv','m3_tap']
    empties = empties.rename({0:"m3_cv",1:"m3_tap"}, axis='columns') 
    empties.set_index('datetime',inplace=True)
    # Add the rows with NaN 
    sumday = sumday.append(empties)
    # delete the values from the original data frame of which daily sum has been calculated now
    print( df[~df.index.isin(sumday.index)])
    df = df[~df.index.isin(sumday.index)]
    
    
    df = df.append(sumday)
    df.sort_index(inplace=True,axis=0)
    dowrite=df.to_csv(fn)
    
    
    # Get the knmi data daily granularity
    df_hdd = gethdd.retrieveHDD()
    df_hdd = df_hdd[df.index.min():]
    df = df.merge(df_hdd,left_on="datetime",right_on="datetime",how='right')

    # Exclude the difference in datapoints between KNMI & gas, if any
    # for example, als je een dag uit het verleden plot, behoeft de data niet up to date te zijn. 
    # 
    df = df[df['m3_cv'].isnull()==False]

    gn_settings = GetMySQL.readsettings()

    dag = sumday.index[0].date()
    enddag = sumday.index.max().date()
    nxtdag = dag + pd.Timedelta(hours = 24)

    while dag <=  enddag:
            #print(dag)
            # plot day + cons ~ hdd van preceding days.
            # need the df for dag
            # need df_hdd for all days distinct last nr days in the function

            nxtdag = dag + pd.Timedelta(hours = 24)
            print("\n")
            print(dag)
            dfdummy = df_GandT[(df_GandT.index.date >= dag) & (df_GandT.index.date < nxtdag)]
            if not dfdummy.empty:
                a = plot_Data(df_GandT[dag:nxtdag],df,True,True,gn_settings['drpbox'])
    #         plotconsumption(df_GandT[dag:nxtdag],dag,df_hdd,showthegraph)

            #stap naar volgende dag
            dag = dag + pd.Timedelta(hours = 24)
    
#     result1,result1_d,result2,ds = plot_dailyconsumption("Gisteren", True)

POI is :  {'start': '2021-12-25', 'end': '2022-04-27'}
At least one datapoint is not in file : Need update
               dy     ratio    var
2021-12-26  851.0  0.590972  CVout
2022-02-19  798.0  0.554167  CVout
2022-02-20    0.0  0.000000  CVout
2022-02-21    0.0  0.000000  CVout
2022-02-22    0.0  0.000000  CVout
2022-02-23    0.0  0.000000  CVout
2022-02-24    0.0  0.000000  CVout
2022-02-25    0.0  0.000000  CVout
2022-02-26    0.0  0.000000  CVout
2022-02-27    0.0  0.000000  CVout
2022-02-28    0.0  0.000000  CVout
2022-03-01  126.0  0.087500  CVout
2022-03-26  869.0  0.603472  CVout
2022-03-27    0.0  0.000000  CVout
2022-03-28    0.0  0.000000  CVout
2022-03-29    0.0  0.000000  CVout
2022-03-30    0.0  0.000000  CVout
2022-03-31    0.0  0.000000  CVout
2022-04-01    0.0  0.000000  CVout
2022-04-02  562.0  0.390278  CVout
 Dates that do NOT meet treshold
               dy     ratio       var
2021-12-26  851.0  0.590972  tapwater
2022-02-19  798.0  0.554167  tapwater
2022-02-20 

KeyError: 'cv'

In [21]:
df_hdd

NameError: name 'df_hdd' is not defined